In [1]:
## Importing libraries
import os
import torch
import torchvision
import torch.optim as optim
import argparse
import matplotlib
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import import_ipynb
import data_preprocessing
import gibbs_sampler_poise
import kl_divergence_calculator
from torch.utils.tensorboard import SummaryWriter
import copy
import numpy as np
from tqdm import tqdm
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image
matplotlib.style.use('ggplot')
from torch.nn import functional as F  #for the activation function
import umap
from torchviz import make_dot

#random.seed(4)

importing Jupyter notebook from data_preprocessing.ipynb
importing Jupyter notebook from gibbs_sampler_poise.ipynb
importing Jupyter notebook from kl_divergence_calculator.ipynb


In [2]:
# learning parameters
batch_size = 1
lr = 1e-4
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device=torch.device('cpu')
tx = transforms.ToTensor()

In [3]:
## Importing MNIST and MNIST datasets
joint_dataset_train=data_preprocessing.JointDataset(mnist_pt_path_1="/home/achint/Practice_code/VAE/MNIST/MNIST/processed/training.pt",
                             mnist_pt_path_2="/home/achint/Practice_code/VAE/MNIST/MNIST/processed/training.pt")
joint_dataset_test = data_preprocessing.JointDataset(mnist_pt_path_1="/home/achint/Practice_code/VAE/MNIST/MNIST/processed/test.pt",
                             mnist_pt_path_2="/home/achint/Practice_code/VAE/MNIST/MNIST/processed/test.pt")

joint_dataset_train_loader = DataLoader(
    joint_dataset_train,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True
)
joint_dataset_test_loader = DataLoader(
    joint_dataset_test,
    batch_size=batch_size,
    shuffle=False,
    drop_last=True
)

In [4]:
writer=SummaryWriter('/home/achint/Practice_code/logs')

In [5]:
img1_grid[2].size()

NameError: name 'img1_grid' is not defined

In [5]:
examples= iter(joint_dataset_train_loader)
example1_data,example2_data,example1_labels,example2_labels=examples.next()
img1_grid= torchvision.utils.make_grid(example1_data)
img2_grid= torchvision.utils.make_grid(example2_data)
writer.add_image('mnist1',img1_grid)
writer.add_image('mnist2',img2_grid)

TypeError: Cannot handle this data type: (1, 1, 128), |u1

In [4]:
latent_dim1 = 1
latent_dim2 = 1
dim_MNIST   = 784
class VAE(nn.Module):
    def __init__(self):
        super(VAE,self).__init__()
        self.gibbs                   = gibbs_sampler_poise.gibbs_sampler()  
        self.kl_div                  = kl_divergence_calculator.kl_divergence()
        ## Encoder set1(MNIST)
        self.set1_enc1 = nn.Linear(in_features = dim_MNIST,out_features = 512)
        self.set1_enc2 = nn.Linear(in_features = 512,out_features = 128)
        self.set1_enc3 = nn.Linear(in_features = 128,out_features = 2*latent_dim1)
        ## Encoder set2(MNIST)
        self.set2_enc1 = nn.Linear(in_features = dim_MNIST,out_features = 512)
        self.set2_enc2 = nn.Linear(in_features = 512,out_features = 128)
        self.set2_enc3 = nn.Linear(in_features = 128,out_features = 2*latent_dim2) 
        ## Decoder set1(MNIST)
        self.set1_dec1 = nn.Linear(in_features = latent_dim1,out_features = 128)
        self.set1_dec2 = nn.Linear(in_features = 128,out_features = 512)
        self.set1_dec3 = nn.Linear(in_features = 512,out_features = dim_MNIST)
        ## Decoder set2(MNIST)
        self.set2_dec1 = nn.Linear(in_features = latent_dim2,out_features = 128)
        self.set2_dec2 = nn.Linear(in_features = 128,out_features = 512)
        self.set2_dec3 = nn.Linear(in_features = 512,out_features = dim_MNIST)
        #self.register_parameter(name='g11', param = nn.Parameter(torch.randn(latent_dim1,latent_dim2)))
        #self.register_parameter(name='g22', param = nn.Parameter(torch.randn(latent_dim1,latent_dim2)))
        self.flag_initialize= 1
        self.g11=torch.randn(latent_dim1,latent_dim2).to(device)
        self.g22=torch.randn(latent_dim1,latent_dim2).to(device)        
        self.g12= torch.zeros(latent_dim1,latent_dim2).to(device)
    def forward(self,x1,x2):
        data1    = x1 #MNIST
        data2    = x2 #MNIST
        # Modality 1 (MNIST)
        x1       = F.relu(self.set1_enc1(x1))
        x1       = F.relu(self.set1_enc2(x1))  
        x1       = self.set1_enc3(x1).view(-1,2,latent_dim1)  # ->[128,2,32]
        mu1      = x1[:,0,:] # ->[128,32]
        log_var1 = x1[:,1,:] # ->[128,32]
        var1     = -torch.exp(log_var1)           #lambdap_2<0
        # Modality 2 (MNIST)
        x2       = F.relu(self.set2_enc1(x2))
        x2       = F.relu(self.set2_enc2(x2))  
        x2       = self.set2_enc3(x2).view(-1,2,latent_dim2)  # ->[128,2,32]
        mu2      = x2[:,0,:] # ->[128,32]
        log_var2 = x2[:,1,:] # ->[128,32]
        var2     = -torch.exp(log_var2)           #lambdap_2<0 
        g22      = -torch.exp(self.g22)     
        g11_copy = self.g11.detach()
        g22_copy = g22.detach()
        mu1_copy = mu1.detach()
        mu2_copy = mu2.detach()
        var1_copy=var1.detach()
        var2_copy=var2.detach()   
        
        if self.flag_initialize==1:
            self.flag_initialize=0
            z1_prior,z2_prior = self.gibbs.initialize_prior_sample(g11_copy,g22_copy)
            z1_posterior,z2_posterior = self.gibbs.initialize_posterior_sample(g11_copy,g22_copy,mu1_copy,var1_copy,mu2_copy,var2_copy)
            self.z1_prior =z1_prior
            self.z2_prior =z2_prior
            self.z1_posterior=z1_posterior
            self.z2_posterior=z2_posterior
            print('z1_gibbs_prior',torch.sum(self.z1_prior))
            print('z2_gibbs_prior',torch.sum(self.z2_prior))     
            print('z1_gibbs_posterior',torch.sum(self.z1_posterior))
            print('z2_gibbs_posterior',torch.sum(self.z2_posterior))        
        z1_prior     = self.z1_prior.detach()
        z2_prior     = self.z2_prior.detach()
        z1_posterior = self.z1_posterior.detach()
        z2_posterior = self.z2_posterior.detach()

        self.z1_gibbs_prior,self.z2_gibbs_prior         = self.gibbs.prior_sample(z1_prior,z2_prior,self.g11,g22)
        self.z1_gibbs_posterior,self.z2_gibbs_posterior = self.gibbs.posterior_sample(z1_posterior,z2_posterior,self.g11,g22,mu1,var1,mu2,var2)
        self.z1_posterior = self.z1_gibbs_posterior.detach()
        self.z2_posterior = self.z2_gibbs_posterior.detach()
        self.z1_prior = self.z1_prior.detach()
        self.z2_prior = self.z2_prior.detach()
        G1 = torch.cat((self.g11,self.g12),0)
        G2 = torch.cat((self.g12,g22),0)
        G  = torch.cat((G1,G2),1)
        
        # decoding for MNIST1
        x1 = F.relu(self.set1_dec1(self.z1_gibbs_posterior))
        x1 = self.set1_dec2(x1)
        reconstruction1=torch.sigmoid(self.set1_dec3(x1))
        # decoding for MNIST2
        x2 = F.relu(self.set2_dec1(self.z2_gibbs_posterior))
        x2 = self.set2_dec2(x2)
        reconstruction2=torch.sigmoid(self.set2_dec3(x2))
        # calculating loss
        lambda2 = torch.cat((mu2,var2),1)                         # Output of encoder for set2 
        part_fun0,part_fun1,part_fun2 = self.kl_div.calc(G,self.z1_posterior,self.z2_posterior,self.z1_prior,self.z2_prior,mu1,var1,mu2,var2)

#         print('z1_gibbs_prior',torch.sum(self.z1_gibbs_prior))
#         print('z2_gibbs_prior',torch.sum(self.z2_gibbs_prior))
#         print('z1_gibbs_posterior',torch.sum(self.z1_posterior))
#         print('z2_gibbs_posterior',torch.sum(self.z2_posterior))

        bce_loss = nn.BCELoss(reduction='sum')
        MSE1 = bce_loss(reconstruction1, data1)
        MSE2 = bce_loss(reconstruction2, data2)
        KLD  = part_fun0+part_fun1+part_fun2
#         print('MSE1',MSE1)
#         print('MSE2',MSE2)
#         print('KLD',KLD)
#         make_dot(g22,params=dict(model.named_parameters()),show_attrs=True, show_saved=True).render("g22", format="png")
#         make_dot(self.g11,params=dict(model.named_parameters()),show_attrs=True, show_saved=True).render("g11", format="png")

#         make_dot(part_fun1,params=dict(model.named_parameters()),show_attrs=True, show_saved=True).render("part_fun1", format="png")
#         make_dot(part_fun2,params=dict(model.named_parameters()),show_attrs=True, show_saved=True).render("part_fun2", format="png")
#         make_dot(MSE1,params=dict(model.named_parameters()),show_attrs=True, show_saved=True).render("MSE1", format="png")
#         make_dot(MSE2,params=dict(model.named_parameters()),show_attrs=True, show_saved=True).render("MSE2", format="png")
#         make_dot(self.z1_gibbs_prior,params=dict(model.named_parameters()),show_attrs=True, show_saved=True).render("z1_prior", format="png")
#         make_dot(self.z2_gibbs_prior,params=dict(model.named_parameters()),show_attrs=True, show_saved=True).render("z2_prior", format="png")
#         make_dot(self.z1_gibbs_posterior,params=dict(model.named_parameters()),show_attrs=True, show_saved=True).render("z1_posterior", format="png")
#         make_dot(self.z2_gibbs_posterior,params=dict(model.named_parameters()),show_attrs=True, show_saved=True).render("z2_posterior", format="png")

        
        loss = MSE1+MSE2+KLD


        return reconstruction1,reconstruction2,mu1,var1,mu2,var2,loss

In [5]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(),lr=lr)
for name, para in model.named_parameters():
    print(name)

set1_enc1.weight
set1_enc1.bias
set1_enc2.weight
set1_enc2.bias
set1_enc3.weight
set1_enc3.bias
set2_enc1.weight
set2_enc1.bias
set2_enc2.weight
set2_enc2.bias
set2_enc3.weight
set2_enc3.bias
set1_dec1.weight
set1_dec1.bias
set1_dec2.weight
set1_dec2.bias
set1_dec3.weight
set1_dec3.bias
set2_dec1.weight
set2_dec1.bias
set2_dec2.weight
set2_dec2.bias
set2_dec3.weight
set2_dec3.bias


In [6]:
def train(model,joint_dataloader):
    model.train()
    running_loss = 0.0
    for i,joint_data in enumerate(joint_dataloader):
        data1    = joint_data[0]
        data1    = data1.float()
        data2   = joint_data[1]
        data2   = data2.float()
        data1    = data1.to(device)
        data2   = data2.to(device)
        data1    = data1.view(data1.size(0), -1)
        data2   = data2.view(data2.size(0), -1)
        optimizer.zero_grad()
        reconstruction1,reconstruction2,mu1,var1,mu2,var2,loss       = model(data1,data2)  
        running_loss += loss.item()          #.item converts tensor with one element to number
        loss.backward()                      #.backward
        optimizer.step()                     #.step one learning step
#         for name, para in model.named_parameters():
#             print(torch.sum(para.grad))
    train_loss = running_loss/(len(joint_dataloader.dataset))

    return train_loss
    
def test(model,joint_dataloader):

    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for i,joint_data in enumerate(joint_dataloader):
            data1   = joint_data[0]
            data1   = data1.float()

            data2  =joint_data[1]
            data2 = data2.float()


            data1 = data1.to(device)
            data2 = data2.to(device)
            data1 = data1.view(data1.size(0), -1)
            data2 = data2.view(data2.size(0), -1)
            reconstruction1,reconstruction2,mu1,var1,mu2,var2,loss = model(data1,data2)  

            running_loss += loss.item()
            #save the last batch input and output of every epoch
            if i == int(len(joint_dataloader.dataset)/joint_dataloader.batch_size) - 1:
                num_rows = 8
                both = torch.cat((data1.view(batch_size, 1, 28, 28)[:8], 
                                  reconstruction1.view(batch_size, 1, 28, 28)[:8]))
                bothp = torch.cat((data2.view(batch_size, 1, 28, 28)[:8], 
                                  reconstruction2.view(batch_size, 1, 28, 28)[:8]))
                save_image(both.cpu(), f"/home/achint/Practice_code/Synthetic_dataset/POISE_VAE_MNIST_MNIST/reconstructions/1_outputMNIST_1_{epoch}.png", nrow=num_rows)
                save_image(bothp.cpu(), f"/home/achint/Practice_code/Synthetic_dataset/POISE_VAE_MNIST_MNIST/reconstructions/2_outputMNIST_2_{epoch}.png", nrow=num_rows)

    test_loss = running_loss/len(joint_dataloader.dataset)
    return test_loss

In [7]:
train_loss = []
test_loss = []
epochs = 10
for epoch in range(epochs):
    print(f"Epoch {epoch+1} of {epochs}")
    train_epoch_loss = train(model,joint_dataset_train_loader)
    test_epoch_loss = test(model,joint_dataset_test_loader)
    train_loss.append(train_epoch_loss)
    test_loss.append(test_epoch_loss)     
    print(f"Train Loss: {train_epoch_loss:.4f}")
    print(f"Test Loss: {test_epoch_loss:.4f}")

Epoch 1 of 10
z1_gibbs_prior tensor(-0.6375, device='cuda:0')
z2_gibbs_prior tensor(-0.3703, device='cuda:0')
z1_gibbs_posterior tensor(-0.3411, device='cuda:0')
z2_gibbs_posterior tensor(-5.6917, device='cuda:0')


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.